# Desafio

En este desafío, el servidor permite a un usuario registrarse con un correo electrónico, y devuelve un perfil, opcionalmente cifrado con AES en modo ECB.

Si el usuario registró el correo usuario@example.com , el perfil tendrá la forma:


```
user=usuario@example.com&id=547&role=user
```


Es decir, está compuesto por un conjunto de pares atributo-valor, de la forma <atributo>=<valor>, y separados el carácter &, de la misma forma que en la query string de una URL.

Para obtener el perfil, se debe hacer un requerimiento GET a una URL de la forma:

```
https://ciberseguridad.diplomatura.unc.edu.ar/cripto/ecb-forge/<email>/register?email=<email-a-registrar>
```

dónde <email> debe ser reemplazado por una dirección de correo electrónico registrada, y <email-a-registrar> por la dirección de correo electrónico que se desee registrar.

En el siguiente ejemplo, ejecutado con curl, la dirección que está operando el desafío es user@example.com y la dirección que se desea registrar es usuario@ejemplo.edu.ar:

```
$ curl https://ciberseguridad.diplomatura.unc.edu.ar/cripto/ecb-forge/user@example.com/register?email=usuario@ejemplo.edu.ar
user=usuario@ejemplo.edu.ar&id=207&role=user
```

Si se le pasa un segundo par encrypted=true en la consulta, el servidor devuelve el mismo perfil, cifrado con AES en modo ECB, y codificado en base64.

Veamos el mismo ejemplo:
```
$ curl 'https://ciberseguridad.diplomatura.unc.edu.ar/cripto/ecb-forge/user@example.com/register?email=usuario@ejemplo.edu.ar&encrypted=true'
fpp5oGqLhBuoBntD3eDaOqq2rqWeQjU2AGqdfTyFq96xmniMMSjuYQQkkwGRd4QM
```

El servidor devuelve el perfil cifrado y codificado en base64. Obsérvese que hemos encerrado todo el argumento de curl entre comillas para evitar que el shell interprete el carácter &. Obviamente esta precaución es innecesaria si se utiliza un browser.

El desafío es alterar el mensaje cifrado de manera tal que al descifrarlo incluya el par role=admin. Esta alteración es posible porque el modo ECB es maleable. Es posible intercambiar o insertar bloques en el texto cifrado de manera tal que al descifrarlo el texto claro haya sido modificado de manera predecible.

# Respuesta
La respuesta debe ser enviada mediante un requerimiento POST a la URL

```
https://ciberseguridad.diplomatura.unc.edu.ar/cripto/ecb-forge/<email>/answer
```

El contenido debe ser de tipo FORM, con un campo message con un nuevo mensaje cifrado codificado en base64. Al descifrarlo, debe contener el par atributo-valor role=admin.

#Análisis
La clave para resolver este desafío es comprender que podemos armar un mensaje falsificado combinando bloques que contienen texto seleccionado por nosotros. En este caso, el único texto bajo nuestro control es el correo electrónico, por lo que será necesario enviar distintas direcciones de manera tal que ciertos bloques contengan el texto que nosotros deseamos.

## Puntos a tener en cuenta
* El algoritmo utilizado es AES, por lo que los bloques son de 128 bits (16
 bytes).
* El texto cifrado enviado como respuesta debe descifrarse a una consulta (query string) válida:
  * Debe tener exactamente un campo user, con una dirección de correo válida.
  * Debe contener al menos un campo id.
  * Debe contener al menos un campo role con el valor admin.
  * Puede contener otros pares atributo-valor aparte de los mencionados.
* Como el modo utilizado es ECB, se utiliza relleno (padding). El relleno utilizado es el estandarizado en PKCS7: se aplican tantos bytes como sea necesario para que el mensaje sea múltiplo del tamaño de bloque, y el valor de esos bytes es la longitud del relleno. Si el mensaje original era múltiplo del tamaño de bloque, el relleno es un bloque adicional con todos sus bytes con el valor 16. Al descifrar el mensaje, el relleno debe ser válido.

In [6]:
import requests
import base64

In [14]:
def print_hex(lista, bytes_):
  hex_list = ''
  for each in range(0,len(lista), 2):
    if each % bytes_ == 0:
      print(hex_list)
      hex_list = ''
    hex_list = hex_list + ' ' + lista[each:each+2]
  print(hex_list)


In [9]:
def print_block(lista, bytes_):
  hex_list = ''
  for each in range(0,len(lista)):
    if each % bytes_ == 0:
      print(hex_list)
      hex_list = ''
    hex_list = hex_list + ' ' + lista[each:each+1]
  print(hex_list)

In [115]:
def get_profile(email, email_register):
  url_request = f'https://ciberseguridad.diplomatura.unc.edu.ar/cripto/ecb-forge/{email}/register?email={email_register}'
  # Solicitud
  request = requests.get(url_request)
  # Body de la respuesta
  message = request.content
  print_block(message.decode('ascii'), 16)
  return message

In [116]:
def get_profile_encripted(email, email_register):
  url_request_encrypted = f'https://ciberseguridad.diplomatura.unc.edu.ar/cripto/ecb-forge/{email}/register?email={email_register}&encrypted=true'
  request_encrypted = requests.get(url_request_encrypted)
  message_encrypted = request_encrypted.content
  message_encrypted_decoded = b64_decode(message_encrypted)
  print_hex(message_encrypted_decoded, 32)
  return message_encrypted_decoded

In [117]:
def b64_decode(message_encrypted):
  message_decoded_from_base64 = base64.b64decode(message_encrypted)
  return message_decoded_from_base64.hex()

In [118]:
email = 'solujan@gmail.com'
email_register = 'adminadnmi@bbb'
message = get_profile(email, email_register)


 u s e r = a d m i n a d n m i @
 b b b & i d = 9 2 1 & r o l e =
 u s e r


In [119]:
# Solicitud
message_hex = get_profile_encripted(email, email_register)



 08 25 42 43 e0 6b 80 de be f0 89 60 1d d6 88 83
 e3 06 c7 19 e6 f4 64 0d c4 ee 01 2d 92 69 95 eb
 85 46 1c 48 1e 38 f6 e6 b9 e1 27 0a 52 37 71 f6


In [120]:
email_register_2 = "adminadnmi@userbbbbbbbbbbbbadmin"
message_2 = get_profile(email, email_register_2)



 u s e r = a d m i n a d n m i @
 u s e r b b b b b b b b b b b b
 a d m i n & i d = 1 3 6 & r o l
 e = u s e r


In [121]:
message_hex_2 = get_profile_encripted(email, email_register_2)



 08 25 42 43 e0 6b 80 de be f0 89 60 1d d6 88 83
 ee 42 74 e0 3a 26 f3 09 60 28 02 0f 0b 8b ee dd
 ad 42 87 01 60 83 94 6a 71 eb f1 09 f6 d3 39 7e
 f2 be 92 d2 b4 7c 7f 73 5f 8e 82 53 f1 f4 e6 c3


In [122]:
email_register_3 = "adminadnmi@usebbbbbbbadmin"
message_3 = get_profile(email, email_register_3)


 u s e r = a d m i n a d n m i @
 u s e b b b b b b b a d m i n &
 i d = 4 4 3 & r o l e = u s e r


In [123]:
message_hex_3 = get_profile_encripted(email, email_register_3)



 08 25 42 43 e0 6b 80 de be f0 89 60 1d d6 88 83
 0c cf 51 3d 71 1a 6b a7 13 dd 3b d6 5c 4e a7 f3
 5e 6b f2 34 cd d2 40 53 07 d8 74 56 5a dd 85 de
 38 16 9f a9 6e 5f 61 be 62 24 44 56 f7 12 48 52


## Rearmo los bloques

In [125]:
new_message = message[:32] + message_2[32:32+16] + message[:16]
print_block(new_message.decode('ascii'), 16)


 u s e r = a d m i n a d n m i @
 b b b & i d = 9 2 1 & r o l e =
 a d m i n & i d = 1 3 6 & r o l
 u s e r = a d m i n a d n m i @


In [126]:
admin_replace = message_hex[:64] + message_hex_2[64:64+32] + message_hex[:32] + message_hex_3[64+32:]
admin_replace

'08254243e06b80debef089601dd68883e306c719e6f4640dc4ee012d926995ebad4287016083946a71ebf109f6d3397e08254243e06b80debef089601dd6888338169fa96e5f61be62244456f7124852'

In [127]:
print_hex(admin_replace, 32)


 08 25 42 43 e0 6b 80 de be f0 89 60 1d d6 88 83
 e3 06 c7 19 e6 f4 64 0d c4 ee 01 2d 92 69 95 eb
 ad 42 87 01 60 83 94 6a 71 eb f1 09 f6 d3 39 7e
 08 25 42 43 e0 6b 80 de be f0 89 60 1d d6 88 83
 38 16 9f a9 6e 5f 61 be 62 24 44 56 f7 12 48 52


In [131]:
b64 = base64.b64encode(bytes.fromhex(admin_replace)).decode()
b64

'CCVCQ+BrgN6+8IlgHdaIg+MGxxnm9GQNxO4BLZJpleutQocBYIOUanHr8Qn20zl+CCVCQ+BrgN6+8IlgHdaIgzgWn6luX2G+YiREVvcSSFI='

In [ ]:
# curl -F message=CCVCQ+BrgN6+8IlgHdaIg+MGxxnm9GQNxO4BLZJpleutQocBYIOUanHr8Qn20zl+CCVCQ+BrgN6+8IlgHdaIgzgWn6luX2G+YiREVvcSSFI= https://ciberseguridad.diplomatura.unc.edu.ar/cripto/ecb-forge/solujan@gmail.com/answer
